<a href="https://colab.research.google.com/github/arshpreetsingh134/Social-Distancing-Tool/blob/master/social_dist_tool.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Install package**

In [ ]:
# Torch is a Tensor library like NumPy, with strong GPU support
# Torchvision is a library for Computer Vision that goes hand in hand with Torch
# CUDA is a parallel computing platform and API that allows tensor computations on an NVIDIA GPU.
# COCO (Common Object in Context) is a large-scale object detection, segmentation, and captioning dataset.
# COCO PythonAPI provides Python API that assists in loading, parsing, and visualizing the annotations in COCO.
# Detectron2 is Facebook AI Research's library that provides state-of-the-art object detection and segmentation algorithms. 
# Opencv is pre-installed on colab
# FiftyOne is an open source ML tool created by Voxel51 that helps you build high-quality datasets and computer vision models.
!pip install -U 'git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI'
!pip install pyyaml==5.1
!pip install fiftyone

import torch
TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]
print("torch: ", TORCH_VERSION, "; cuda: ", CUDA_VERSION)
# Install detectron2 that matches the above pytorch version
# See https://detectron2.readthedocs.io/tutorials/install.html for instructions
!pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/$CUDA_VERSION/torch$TORCH_VERSION/index.html

# After installation, you may need to "restart runtime" in Colab.

# **Import libraries**

In [ ]:
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import os, json, cv2, random
from google.colab.patches import cv2_imshow

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog

# import fiftyone
import fiftyone as fo
import fiftyone.zoo as foz

# **People dataset - Get data**

Download 'people' dataset from COCO-2017 using fiftyone (Docs > FiftyOne Integrations > COCO Integration)

In [ ]:
# List available zoo datasets
print(foz.list_zoo_datasets())

# To download the COCO dataset for only the "person" classes
dataset = foz.load_zoo_dataset(
    "coco-2017",
    #split='train' #load train, val and test data by default
    label_types=["detections"],
    classes=["person"],
    only_matching=True,
    max_samples=300,
)

Register a COCO Format Dataset
If your instance-level (detection, segmentation, keypoint) dataset is already a json file in the COCO format, the dataset and its associated metadata can be registered easily with:

In [ ]:
from detectron2.data.datasets import register_coco_instances

dataset_name_list = ('people_train','people_val')

for dataset_name in dataset_name_list:
  if dataset_name in DatasetCatalog.list():
    DatasetCatalog.remove(dataset_name)

register_coco_instances("people_train", {}, "/root/fiftyone/coco-2017/train/labels.json", "/root/fiftyone/coco-2017/train/data")
register_coco_instances("people_val", {}, "/root/fiftyone/coco-2017/validation/labels.json", "/root/fiftyone/coco-2017/validation/data")

# **People dataset - Train Model**

Once you’ve registered the dataset, there are configs you might want to change to train or evaluate on new dataset.

In [ ]:
from detectron2.engine import DefaultTrainer

cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml"))
#faster_rcnn_R_50_C4_1x.yaml - in the orignal Faster R-CNN paper, without FPN
#faster_rcnn_X_101_32x8d_FPN_3x.yaml - better accuracy but longer training time and larger memory usage
cfg.DATASETS.TRAIN = ("people_train",)
cfg.DATASETS.TEST = ("people_val",)
cfg.DATALOADER.NUM_WORKERS = 4
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/faster_rcnn_R_50_C4_1x.yaml")  # Let training initialize from model zoo
cfg.SOLVER.IMS_PER_BATCH = 4
cfg.SOLVER.BASE_LR = 0.0002
cfg.SOLVER.MAX_ITER = 300 #increase if val mAP is still rising, decrease if overfit
cfg.SOLVER.STEPS = [] # do not decay learning rate
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128 #64
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 80
cfg.TEST.EVAL_PERIOD = 500

os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
trainer = DefaultTrainer(cfg) 
trainer.resume_or_load(resume=False)
trainer.train()

In [ ]:
# Look at training curves in tensorboard:
%load_ext tensorboard
%tensorboard --logdir output

# **People dataset - Inference & Performance Evaluation**

In [ ]:
# Inference should use the config with parameters that are used in training
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")  # path to the model we just trained
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.7   # set a testing threshold
predictor = DefaultPredictor(cfg)

Evaluate performance using AP metric implemented in COCO API.

In [ ]:
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader
evaluator = COCOEvaluator("people_val", output_dir="./output")
val_loader = build_detection_test_loader(cfg, "people_val")
print(inference_on_dataset(predictor.model, val_loader, evaluator))

# **Implement Social Distancing check for a video clip**

With Custom Objection Detection Model ready, we will use it in below implementation of 'social distancing for a video clip'

## **Mount drive**



In [ ]:
# Mount drive
from google.colab import drive
drive.mount('/content/gdrive')

# Change work directory
import os
os.chdir('/content/gdrive/My Drive/Social_Distancing')

## **Use pre-trained model**

In [ ]:
# Use pre-trained Custom Object Detection Model
# The model and predictor are already defined as above, can be used directly

# Below codes are used if need to use baseline model instead of custom model
# cfg = get_cfg()
# cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml"))
# cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.7  # threshold to filter out low-scored bounding boxes predicted by the Fast R-CNN 
# cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml")
# predictor = DefaultPredictor(cfg)

## **Read a video and convert to frames/images**

In [ ]:
# Path to video file & images folder
video = "/content/gdrive/MyDrive/My video/sample.mp4"
img_path = "/content/gdrive/MyDrive/Social_Distancing/"

# Capture video
video_cap = cv2.VideoCapture(video)

# Check if video file is opened successfully
if (video_cap.isOpened()== False): 
  print("Error opening video file")

count=0

# Convert video to frames/images

while(video_cap.isOpened()):
    
  # Read frame by frame
  ret, frame = video_cap.read()
  if ret == True:

    # Write each frame to image file and save to folder        
    cv2.imwrite(img_path+str(count)+'.png', frame) # 1.png, 2.png, etc.
    count+=1
    if(count==200): # Limit the no. of frames (otherwise take too long - 25fps)
      break

  else: 
    break

## **Functions to read images, do predictions and process images**



In [ ]:
# Function: returns the bottom center of every person's bb.

def cal_bottom_ctr(img, person, i):

  x1,y1,x2,y2 = person[i]

  x = int((x1+x2)/2) #center
  y = int(y2) #bottom
  
  bottom_ctr = (x, y)

  return bottom_ctr

(x1,y1).................

.................(x2,y2)

In [ ]:
# Function: compute the Euclidian Distance.

from scipy.spatial import distance

def compute_distance(bottom_ctrs,num):
  dist = np.zeros((num,num))
  for i in range(num):
    for j in range(i+1,num):
      dist[i][j] = distance.euclidean(bottom_ctrs[i], bottom_ctrs[j])
  return dist

In [ ]:
# Function: returns people with distance<threshold

def find_violation(dist,num,threshold):
  p1=[]
  p2=[]
  violation=[]
  for i in range(num):
    for j in range(i+1,num):
      if (dist[i][j]<threshold):
        p1.append(i)
        p2.append(j)
        violation.append(dist[i][j])
  return p1,p2,violation

In [ ]:
# Function: change the color of bb to Red for persons who violate social distancing

def add_redbox(img, person, p1, p2):
  #nums = np.unique(p1+p2)
  for i in (p1+p2): # Merge, any person in either p1 or p2
    x1,y1,x2,y2 = person[i]
    _=cv2.rectangle(img, (x1,y1), (x2,y2), (255,0,0), 3) #red box
  return img

In [ ]:
# Function: read image, use model to do prediction, + other functions as above

def find_violation_add_redbox(imgfile_name, threshold):
  img = cv2.imread(img_path+imgfile_name)
  outputs = predictor(img)
  classes = outputs['instances'].pred_classes.cpu().numpy()
  bbox = outputs['instances'].pred_boxes.tensor.cpu().numpy()
  person = bbox[np.where(classes==0)[0]] #choose class='person'
  bottom_ctrs = [cal_bottom_ctr(img, person, i) for i in range(len(person))]
  num = len(bottom_ctrs)
  dist = compute_distance(bottom_ctrs,num)
  p1, p2, violations = find_violation(dist,num,threshold)
  img = add_redbox(img, person, p1, p2)
  cv2.imwrite(img_path+imgfile_name,img)


## **Code execution for all frames/images of a video**

In [ ]:
# The speed is ~1 min for 100 frames/images from the sample video (model = faster_rcnn_R_50_FPN_3x)

import glob
import re

imgfile_names = glob.glob('*.png')
imgfile_names.sort(key=lambda f: int(re.sub('\D', '', f)))

threshold = 100 # Can be fine tuned based on camera calibration
_=[find_violation_add_redbox(imgfile_names[i], threshold) for i in range(len(imgfile_names))]

## **Convert processed & annotated images back to a video**

In [ ]:
# Converting images back to a video

images_all=[]

for i in range(len(imgfile_names)):

  # Read each image file
  img = cv2.imread(imgfile_names[i])
  img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

  # Get image size
  height, width, layers = img.shape
  size = (width,height)

  # Insert the image into an array
  images_all.append(img)

out = cv2.VideoWriter(img_path+'sample_result1.mp4', cv2.VideoWriter_fourcc(*'DIVX'), 25, size)

for i in range(len(imgfile_names)):
  out.write(images_all[i])

out.release()